In [3]:
from bs4 import BeautifulSoup
import html2text

converter = html2text.HTML2Text()

def segment_html(html: str) -> list[str]:
    # Convert markdown to HTML
    # Parse HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Iterate through the elements and associate headers with content
    headers = ['title', 'h1', 'h2']
    segments = []
    current_header = None
    for element in soup.children:
        if element.name in headers:
            current_header = element.getText()
            segments.append(element.getText())
        elif current_header:
            content = converter.handle(str(element))
            segments[-1] = segments[-1] + " " + content
    return segments

In [4]:
import fitz

def classify_text(text, font_size):
    if font_size >= 18:
        return f"<h1>{text}</h1>"
    else:
        return f"<p>{text}</p>"

def pdf_to_html_with_headers(pdf_path):
    doc = fitz.open(pdf_path)
    html_content = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if block["type"] == 0:
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        
                        # Skip spans that are just a bullet point
                        if text == "-":
                            continue

                        font_size = span["size"]
                        html_text = classify_text(text, font_size)
                        html_content.append(html_text)

    doc.close()
    return "".join(html_content)




pdf_path = 'notes.pdf'
html_output = pdf_to_html_with_headers(pdf_path)
print(segment_html(html_output))

with open('output.html', 'w') as html_file:
    html_file.write(html_output)

['Anatomy Course Notes', 'Introduction to Anatomy Definition:\n\n \n\nStudy of the structure of body parts and their relationships.\n\n \n\nBranches:\n\n \n\nGross/Macroscopic, Microscopic, Developmental\n\n', 'The Skeletal System \n\nFunctions:\n\n \n\nSupport, movement, protection, blood cell production, calcium storage.\n\n \n\nBone Types:\n\n \n\nLong (femur), Short (carpals), Flat (sternum), Irregular (vertebrae)\n\n', 'Muscular System \n\nMuscle Types:\n\n \n\nSkeletal (voluntary movement), Smooth (organ walls), Cardiac (heart).\n\n \n\nFunctions:\n\n \n\nMovement, posture, joint stability, heat production.\n\n', 'Nervous System \n\nComponents:\n\n \n\nBrain, spinal cord, nerves.\n\n \n\nFunctions:\n\n \n\nRegulates and controls body functions, sensory response, motor response.\n\n']


In [6]:
%pip install redis

   ---------------------------------------- 0.0/250.3 kB ? eta -:--:--
   ------------------------------- -------- 194.6/250.3 kB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 250.3/250.3 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import redis
from redis.commands.search.field import TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

r = redis.Redis(
  host='redis-17885.c29105.us-east-1-mz.ec2.cloud.rlrcp.com',
  port=17885,
  password='zKPMCEBwHjoAqRyLV2YO8ZuqcQeLqBS9')


schema = (
    TextField("$.user", as_name="user"),
    TextField("$.topic", as_name="topic"),
    TextField("$.text", as_name="text"),
)

index = r.ft("idx:notes")
# index.create_index(
#     schema,
#     definition=IndexDefinition(prefix=["notes:"], index_type=IndexType.JSON),
# )


In [6]:
from redis.commands.search.query import Query

doc_id = "notes:123"
r.json().set(doc_id, '$', {"user": "joe", "topic": "example topic", "text": "example text"})

res = index.search(Query('@topic:"example topic"'))

In [14]:
import json
test = json.loads(res.docs[0].json)
test["text"]

'example text'

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [3]:
import cohere
co = cohere.Client(COHERE_API_KEY)

In [4]:
message = "Hello! How are you?"
response = co.chat(
	message, 
	model="command", 
	temperature=0.1
)

answer = response.text
answer

"Hi there! I'm an AI language model and I don't have emotions as humans do, but I'm ready to assist you with any questions or discussions you have today. How can I help you? \n\nIf you'd like, we can start with a topic or question, or you can just talk to me about anything that's on your mind, and I'll do my best to provide helpful responses. \n\nWould you like to get started with a particular topic or question?"

'01001000 01100101 01101100 01101100 01101111 00100001 00100000 01001000 01101111 01110111 00100000 01100011 01100001 01101110 00100000 01001001 00100000 01101000 01100101 01101100 01110000 00100000 01110100 01101111 01110101 01101100 01110000 00100000 01110100 01101000 01100101 01101101 00100000 01110100 01101000 01100001 01110110 01100101 00100000 01110100 01101000 01100101 01110010 00100000 01100011 01101111 01110101 00100000 01110100 01101000 01100101 01101100 00100000 01110100 01101000 01100001 01110100 00100000 01100010 01101001 01101110 01100001 01110010 00101110.\n\nDid you know that the string "Hello, World!" in binary is the same as the ASCII decimal number 48656c6c6f20776f726c6421 and in hexadecimal it is the same as the bytes 68656c6c6f20776f726c64 and the words "Hello World" written in octal is the same as the bytes 456c6f206c6f72656d20576f726c64, and in quaternary it is the same as the bytes 3078206c696b6520776f726c64? \n\nWould you like me to convert any other strings in

In [31]:
%pip install streamlit


  Using cached streamlit-1.30.0-py2.py3-none-any.whl.metadata (8.2 kB)
  Using cached altair-5.2.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached blinker-1.7.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached cachetools-5.3.2-py3-none-any.whl.metadata (5.2 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 61.2/61.2 kB ? eta 0:00:00
  Using cached pillow-10.2.0-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached protobuf-4.25.2-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached pyarrow-14.0.2-cp311-cp311-win_amd64.whl.metadata (3.1 kB)
  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached tzlocal-5.2-py3-none-any.whl.met

In [32]:
import streamlit as st

# Example flashcards data
flashcards = [
    {"question": "What is the capital of France?", "answer": "Paris"},
    {"question": "What is 2+2?", "answer": "4"},
    # Add more flashcards here
]

# Initialize session state
if 'current_card' not in st.session_state:
    st.session_state.current_card = 0

# Display the current flashcard
st.write(f"Q: {flashcards[st.session_state.current_card]['question']}")

# Button to show answer
if st.button("Show Answer"):
    st.write(f"A: {flashcards[st.session_state.current_card]['answer']}")

# Next card button
if st.button("Next Card"):
    st.session_state.current_card = (st.session_state.current_card + 1) % len(flashcards)


2024-01-13 15:24:09.286 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


AttributeError: st.session_state has no attribute "current_card". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization

In [8]:
context = "Skull: a framework of bone or cartilage enclosing the brain of a vertebrate; the skeleton of a person's or animal's head.\n Dragon: A mythical creature"
topic = "anatomy"

message = f"""You are a professor who is an expert in the field of {topic}. Given the following context related to {topic}, find important terms and definitions that you would ask your students to test their understanding of the topic. You can assume that your students have a basic understanding of the topic.
    Provide a key term and its definition. Return your response in the provided format. Please denote the end of definition by appending [END] to the end of your solution:
    

    Example Response Format (DO NOT INCLUDE AS CONTEXT IN YOUR RESPONSE):

    Term: Femur

    Definition: The bone of the thigh or upper hind limb, articulating at the hip and the knee. [END]


    Context: {context}
    
"""

print(message)

response = co.chat(
	message, 
	model="command", 
	temperature=0.0
)

answer = response.text
answer


You are a professor who is an expert in the field of {topic}. Given the following context related to {topic}, find important terms and definitions that you would ask your students to test their understanding of the topic. You can assume that your students have a basic understanding of the topic.
    Provide a key term and its definition. Return your response in the provided format. Please denote the end of definition by appending [END] to the end of your solution:
    

    Example Response Format (DO NOT INCLUDE AS CONTEXT IN YOUR RESPONSE):

    Term: Femur

    Definition: The bone of the thigh or upper hind limb, articulating at the hip and the knee. [END]


    Context: {context}
    



'Term: Electromagnetic spectrum\n\nDefinition: The electromagnetic spectrum refers to the range of all types of electromagnetic radiation, ranging from low-frequency radio waves to high-frequency gamma rays. These waves differ in their wavelengths and frequencies, but they all share the common characteristic of being produced by the oscillation of electromagnetic charges.\n\nRadio waves, for instance, are utilized in communication technologies such as radio and television broadcasting, while visible light is the spectrum portion that is visible to the human eye and responsible for the colors we perceive. On the other hand, high-energy electromagnetic radiation such as X-rays and gamma rays find applications in medical imaging and cancer treatment, respectively.\n\nUnderstanding the electromagnetic spectrum is crucial as it underpins many modern technologies and facilitates our understanding of how electromagnetic radiation interacts with matter. [END]\n\nWould you like me to define ano